In [4]:
# pip install requests beautifulsoup4 pandas selenium
# Importing necessary libraries
import requests  # For fetching the webpage
from bs4 import BeautifulSoup  # For parsing the webpage
# URL of the webpage to scrape
url = 'https://starbucksmenuprices.com/'

# Send a request to the server
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Successfully fetched the webpage!")
else:
    print(f"Failed to fetch webpage. Status code: {response.status_code}")

# Parse the webpage content with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Print the HTML content to understand its structure
# print(soup.prettify())

# Find all <ul> elements containing the country links
sections = soup.find_all('ul')  # Locate all <ul> elements

# Lists in Python: The result is a list, which can store multiple items.
print(sections[1])

# Example of List:
my_list = ['apple', 'banana', 'cherry']
print(my_list[0],my_list[1],my_list[2])

# Extract links from the <ul> sections
country_links = []  # Empty list to store results

for section in sections:
    links = section.find_all('a')  # Find all <a> tags in each section
    for link in links:
        country_name = link.text.strip()  # Get the visible text of the link
        country_url = link.get('href')  # Get the href attribute (URL)
        country_links.append({'Country': country_name, 'URL': country_url})

# Example of Loop:
for fruit in ['apple', 'banana', 'cherry']:
    print(fruit)

# Import pandas for working with data
import pandas as pd

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(country_links)

# Display the first few rows of the DataFrame
print(df.head())

# Save the DataFrame to a CSV file
df.to_csv('starbucks_country_links.csv', index=False)
print("Saved country links to starbucks_country_links.csv")

# Step 1: Load the Links
links_file = 'starbucks_country_links.csv'  # File containing country links
country_links = pd.read_csv(links_file)  # Read the CSV file into a DataFrame

# Step 2: Filter Valid Link
valid_links = country_links[~country_links['URL'].str.contains('#', na=False)]  # Filter rows without '#'
if valid_links.empty:
    print("No valid links found in the file.")
    exit()

first_link = valid_links.iloc[1]  # Select the first valid row
country_url = first_link['URL']  # Extract the URL from the first valid row
country_name = first_link['Country'].lower().replace(' ', '-')  # Extract and format the country name

# Step 3: Fetch the Webpage Content
response = requests.get(country_url)
if response.status_code == 200:
    print(f"Successfully fetched the page: {country_url}")
else:
    print(f"Failed to fetch webpage. Status code: {response.status_code}")
    exit()

#Step 4: Parse the HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Step 5: Locate "h2" Section
starbucks_prices_heading = soup.find('h2')  # Locate the "h2" section heading
if starbucks_prices_heading:
    starbucks_prices_table = starbucks_prices_heading.find_parent('table')  # Find the parent table containing the data
else:
    print("'Hot Coffee' section not found on the page.")
    exit()

# Step 6: Extract Data
starbucks_prices_data = []  # List to store extracted data
if starbucks_prices_table:
    rows = starbucks_prices_table.find_all('tr', class_='item')  # Find all rows with class "item"
    for row in rows:
        cols = row.find_all('td')  # Find all columns in the row
        cols = [col.string.strip() for col in cols]  # Clean the text
        if cols:  # Skip empty rows
            starbucks_prices_data.append(cols)

# Step 7: Save Data
if starbucks_prices_data:
    df_starbucks_prices= pd.DataFrame(starbucks_prices_data, columns=['Item', 'Price'])  # Create a DataFrame
    output_file = f'starbucks_prices_{country_name}.csv'  # Generate file name with country name
    df_starbucks_prices.to_csv(output_file, index=False)  # Save to CSV
    print(f"Saved 'Starbucks Prices' prices to {output_file}")
    print(df_starbucks_prices)  # Print the result
else:
    print("No 'Starbucks Prices Data' data found.")


import os
import pandas as pd

# Step 1: Define the folder path containing the CSV files
folder_path = './'  # Adjust to the directory where the files are stored

# Step 2: Initialize a list to store latte prices data
worldwide_latte_prices = []

# Step 3: Loop through files in the folder
for file_name in os.listdir(folder_path):
    # Check if the file matches the pattern "starbucks_prices_[country].csv"
    if file_name.startswith('starbucks_prices_') and file_name.endswith('.csv'):
        # Extract the country name from the file name
        country_name = file_name.replace('starbucks_prices_', '').replace('.csv', '').capitalize()
        
        # Load the CSV file into a DataFrame
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)
        
        # Step 4: Search for a row containing 'Latte'
        for _, row in df.iterrows():
            if 'latte' in row['Item'].lower():  # Case-insensitive search for "latte"
                latte_price = row['Price']
                # Append the country name and latte price to the list
                worldwide_latte_prices.append({'Country': country_name, 'Latte Price': latte_price})
                break  # Stop after finding the first matching "latte"

# Step 5: Save the consolidated data to a new CSV file
output_file = 'worldwide_latte_prices.csv'
df_worldwide = pd.DataFrame(worldwide_latte_prices)
df_worldwide.to_csv(output_file, index=False)

print(f"Worldwide latte prices saved to {output_file}")
print(df_worldwide)

# Import necessary libraries
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

# Step 1: Load country data from the CSV file
csv_file = 'worldwide_latte_prices.csv'  # Replace with your CSV file name
countries = pd.read_csv(csv_file)


# Step 2: Set up Selenium WebDriver
driver = webdriver.Chrome()  # Ensure ChromeDriver is properly set up

# Define a function to fetch exchange rate using DuckDuckGo
def get_exchange_rate_from_duckduckgo(country_name):
    try:
        # Open DuckDuckGo and search for the exchange rate
        driver.get("https://duckduckgo.com/")
        search_box = driver.find_element(By.NAME, "q")
        search_box.clear()
        search_box.send_keys(f"{country_name} currency to USD")
        search_box.send_keys(Keys.RETURN)

        # Wait for results to load
        time.sleep(3)

        # Locate the input field containing the exchange rate
        rate_element = driver.find_element(By.CSS_SELECTOR, 'input[aria-label="Currency value in USD"]')  # Adjusted selector
        exchange_rate = rate_element.get_attribute("value")  # Extract the value from the input field
        return exchange_rate
    except Exception as e:
        print(f"Failed to get exchange rate for {country_name}: {e}")
        return None

# Step 3: Loop through each country and fetch exchange rates
exchange_rates = []
for _, row in countries.iterrows():
    country_name = row['Country']
    print(f"Fetching exchange rate for {country_name}...")
    rate = get_exchange_rate_from_duckduckgo(country_name)
    exchange_rates.append({'Country': country_name, 'Exchange Rate': rate})

# Step 4: Save the results to a new CSV file
df_exchange_rates = pd.DataFrame(exchange_rates)
output_file = 'exchange_rates_duckduckgo.csv'
df_exchange_rates.to_csv(output_file, index=False)
print(f"Saved exchange rates to {output_file}")

# Close the Selenium WebDriver
driver.quit()


Successfully fetched the webpage!
<ul class="sub-menu">
<li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-32" id="menu-item-32"><a href="https://starbucksmenuprices.com/starbucks-au-prices/">Australia</a></li>
<li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-42" id="menu-item-42"><a href="https://starbucksmenuprices.com/starbucks-brasil-precos/">Brasil</a></li>
<li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-70" id="menu-item-70"><a href="https://starbucksmenuprices.com/starbucks-%d1%86%d0%b5%d0%bd%d0%b8/">Bulgaria</a></li>
<li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-58" id="menu-item-58"><a href="https://starbucksmenuprices.com/starbucks-canada-menu/">Canada</a></li>
</ul>
apple banana cherry
apple
banana
cherry
     Country                                                URL
0        A-C                                                  #
1  Australia  https://starbu